## Data Shippers

#### Beats is the platform for single-purpose data shippers. They send data from hundreds or thousands of machines and systems to Logstash or Elasticsearch.

#### In this module we will guide the user through the installation of the beats listed below which allos the  collection of various types of host data:

* Filebeat
* Packetbeat
* Metricbeat
* Auditbeat
* Heartbeat

### Filebeat installation

In [ ]:
%%bash
sudo apt install -y filebeat

### Disable default configuration

In [ ]:
%%bash
sudo sed -i '28 s/^/#/' /etc/filebeat/filebeat.yml
sudo sed -i '117 s/^/#/' /etc/filebeat/filebeat.yml
sudo sed -i '148 s/^/#/' /etc/filebeat/filebeat.yml
sudo sed -i '150 s/^/#/' /etc/filebeat/filebeat.yml
sudo sed -i '176,178 s/^/#/' /etc/filebeat/filebeat.yml

### Enable Configuration to Communicate Filebeat with Logstash

##### As we want to establish a connection between filebeat and logstash, we must recall that we have secured all communication with logstash using ssl. Therefore, any beat that seeks to communicate with logstash must provide the certificate we have generated in the logstash node.

##### In this node, the certificate is located in "/etc/pki/tls/certs/logstash-forwarder.crt". Before setting up the necessary configuration for filebeat to connect with logstash, we will first validate if we can stablish a connection to logstash using the following command:

~~~
telnet <LOGSTASH-IP-ADDRESS> 5044
~~~

##### You will get a message stating "Connected to ....."  To exit press CTRL+] and then quit or CTRL+D

##### Now let us validate teh certificate by using the following command:

~~~
curl -v --cacert /etc/pki/tls/certs/logstash-forwarder.crt <LOGSTASH-IP-ADDRESS>:5044
~~~

##### Let us now proceed to set the log files we would like filebeat to track and ship to logstash, set the ip address and port in which logstash will receive our data, and define the location of our certificate that will be used to secure communication between filebeat and logstash.

In [ ]:
%%bash

sudo sed -i "28i \ \ \ \ - /var/log/auth.log" /etc/filebeat/filebeat.yml
sudo sed -i "29i \ \ \ \ - /var/log/syslog" /etc/filebeat/filebeat.yml

sudo sed -i '160 s/^#//g' /etc/filebeat/filebeat.yml 

export ls_ip="<LOGSTASH-IP-ADDRESS>"

sudo sed -i "162i \ \ hosts: [\"$ls_ip:5044\"]" /etc/filebeat/filebeat.yml

sudo sed -i '167i \ \ ssl.certificate_authorities: ["/etc/pki/tls/certs/logstash-forwarder.crt"]' /etc/filebeat/filebeat.yml

### Enabling the Filebeat's system module.

##### Filebeat supports different modules that allows us to collect and parse logs created by Ubuntu's system loggin service. To enable the system module, run the command below:

In [ ]:
%%bash
sudo filebeat modules enable system

##### You can verify if the system module has been enables by using the following command:

~~~
sudo filebeat modules list
~~~

### Loading Index templates

##### An index is a collection of documents that share similar characteristics. We will load an index template for documents stored in filebeat indexes that will be applied automatically when a new index is created. Let us list the current templates in elasticsearch by using the following command:

~~~
curl -u elastic:elasticsiem -XGET '<ELASTICSEARCH-IP-ADDRESS>:9200/_template?filter_path=*.version&pretty'
~~~

In [ ]:
%%bash
curl -u elastic:elasticsiem -XGET '<ELASTICSEARCH-IP-ADDRESS>:9200/_template?filter_path=*.version&pretty'

##### As we can observe, there are no templates defined for filebeat indexes. Let us load the filebeat template into elasticsearch by using the following command:

In [ ]:
%%bash
cd ~
sudo /usr/share/filebeat/bin/filebeat export template > filebeat.template.json -c /etc/filebeat/filebeat.yml
curl -u elastic:elasticsiem -H 'Content-Type: application/json' -XPUT '<ELASTICSEARCH-IP-ADDRESS>:9200/_template/filebeat-7.3.0' -d@/home/ubuntu/filebeat.template.json
curl -u elastic:elasticsiem -XGET '<ELASTICSEARCH-IP-ADDRESS>:9200/_template/' | jq 'keys'
    

### Installing Kibana dashboards for Filebeat

In [ ]:
%%bash
sudo filebeat setup -e \
  -E output.logstash.enabled=false \
  -E output.elasticsearch.hosts=['<ELASTICSEARCH-IP-ADDRESS>:9200'] \
  -E setup.kibana.host=<KIBANA-IP-ADDRESS>:5601 \
  -E output.elasticsearch.username=elastic \
  -E output.elasticsearch.password=elasticsiem

### Start Filebeat

In [ ]:
%%bash
sudo systemctl start filebeat

### Alternatively, you can run Filebeat in debug mode by using the following command:

~~~
sudo /usr/share/filebeat/bin/filebeat -e -d "publish" -c /etc/filebeat/filebeat.yml
~~~

### Backloging in filebeat


##### In teh case that you change some of the filters that you set on logstash and would like to pre-process your data again, you will need to delete all indexes where your data was stored, stop filebeat, delete filebeat's registry file, and start filebeat again.

##### To delete filebeat registry, you can use the following command:

~~~
sudo rm -R /var/lib/filebeat/registry
~~~

##### Note: Whenever you process again your data, your receiving logs will have a timestamp stating the time the log was received by elasticsearch and not the time at which the log was generated. The timestamp can be replaced by adding a rule on logstash that takes the timestamp stated at the beggining of each log and replace teh default timestamp (receiving time) set by elasticsearch.


### Packetbeat Installation

In [ ]:
%%bash
sudo apt install -y packetbeat

### Disable default configuration

##### The default packetbeat configuration enables monitoring certain transaction porotocols such as icmp, ampq, cassandra, among others. We will disable unwanted protocols by commenting the protocol and list of ports and monitor only the following: icmp, dhcpv4, dns, http, and tls.

In [5]:
%%bash
sudo sed -i 's/^\- type\: amqp/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[5672\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: cassandra/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[9042\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: memcache/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[11211\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: mysql/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[3306,3307\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: pgsql/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[5432\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: redis/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[6379\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: thrift/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[9090\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: mongodb/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[27017\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: nfs/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[2049\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^output.elasticsearch\:/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ hosts\: \[\"localhost\:9200\"\]/#&/' /etc/packetbeat/packetbeat.yml

### Enable communication with logstash

In [7]:
%%bash
export ls_ip="<LOGSTASH-IP-ADDRESS>"

#Enable Logstash output
sudo sed -i '/output.logstash\:/s/^#//g' /etc/packetbeat/packetbeat.yml

#Uncomment and Change Logstash IP ADDRESS
sudo sed -i "s/#hosts\: \[\"localhost\:5044\"\]/\ hosts\: \[\"$ls_ip\:5044\"\]/g" /etc/packetbeat/packetbeat.yml
sudo sed -i 's/ #ssl\.certificate_authorities\: \["\/etc\/pki\/root\/ca\.pem"\]/\ ssl\.certificate\_authorities\: \["\/etc\/pki\/tls\/certs\/logstash\-forwarder\.crt"\]/g' /etc/packetbeat/packetbeat.yml


### Test Configuration and Connectivity to Logstash

In [8]:
%%bash
sudo /usr/share/packetbeat/bin/packetbeat test config -c /etc/packetbeat/packetbeat.yml
sudo /usr/share/packetbeat/bin/packetbeat test output -c /etc/packetbeat/packetbeat.yml

Config OK
logstash: 10.1.1.13:5044...
  connection...
    parse host... OK
    dns lookup... OK
    addresses: 10.1.1.13
    dial up... OK
  TLS...
    security: server's certificate chain verification is enabled
    handshake... OK
    TLS version: TLSv1.2
    dial up... OK
  talk to server... OK


### Add Kibana Dashboards for Packetbeat

In [16]:
%%bash
sudo packetbeat setup -e \
  -E output.logstash.enabled=false \
  -E output.elasticsearch.hosts=['<ELASTICSEARCH-IP-ADDRESS>:9200'] \
  -E setup.kibana.host=<KIBANA-IP-ADDRESS>:5601 \
  -E output.elasticsearch.username=elastic \
  -E output.elasticsearch.password=elasticsiem

Index setup finished.
Loading dashboards (Kibana must be running and reachable)
Loaded dashboards


2019-08-07T04:42:29.537Z	INFO	instance/beat.go:606	Home path: [/usr/share/packetbeat] Config path: [/etc/packetbeat] Data path: [/var/lib/packetbeat] Logs path: [/var/log/packetbeat]
2019-08-07T04:42:29.537Z	INFO	instance/beat.go:614	Beat ID: ead067ec-ba6d-4134-a16a-8d89f8e626cf
2019-08-07T04:42:29.538Z	INFO	[beat]	instance/beat.go:902	Beat info	{"system_info": {"beat": {"path": {"config": "/etc/packetbeat", "data": "/var/lib/packetbeat", "home": "/usr/share/packetbeat", "logs": "/var/log/packetbeat"}, "type": "packetbeat", "uuid": "ead067ec-ba6d-4134-a16a-8d89f8e626cf"}}}
2019-08-07T04:42:29.538Z	INFO	[beat]	instance/beat.go:911	Build info	{"system_info": {"build": {"commit": "6f0ec01a0e57fe7d4fd703b017fb5a2f6448d097", "libbeat": "7.3.0", "time": "2019-07-24T17:32:57.000Z", "version": "7.3.0"}}}
2019-08-07T04:42:29.539Z	INFO	[beat]	instance/beat.go:914	Go runtime info	{"system_info": {"go": {"os":"linux","arch":"amd64","max_procs":2,"version":"go1.12.4"}}}
2019-08-07T04:42:29.539Z	INF

### Add template from packetbeat to elasticsearch

In [11]:
%%bash

sudo packetbeat setup --index-management \
  -E output.logstash.enabled=false \
  -E 'output.elasticsearch.hosts=["<ELASTICSEARCH-IP-ADDRESS>:9200"]' \
  -E output.elasticsearch.username=elastic \
  -E output.elasticsearch.password=elasticsiem


Index setup finished.


### List Loaded Templates

In [12]:
%%bash
curl -u elastic:elasticsiem -XGET '<ELASTICSEARCH-IP-ADDRESS>:9200/_template/' | jq 'keys'

[
  ".data-frame-internal-1",
  ".data-frame-notifications-1",
  ".kibana_task_manager",
  ".logstash-management",
  ".management-beats",
  ".ml-anomalies-",
  ".ml-config",
  ".ml-meta",
  ".ml-notifications",
  ".ml-state",
  ".monitoring-alerts-7",
  ".monitoring-beats",
  ".monitoring-es",
  ".monitoring-kibana",
  ".monitoring-logstash",
  ".triggered_watches",
  ".watch-history-9",
  ".watches",
  "filebeat-7.3.0",
  "packetbeat-7.3.0"
]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  150k  100  150k    0     0  6553k      0 --:--:-- --:--:-- --:--:-- 6553k


### Start Packetbeat

In [9]:
%%bash
sudo systemctl start packetbeat

### Alternatively, you can run Filebeat in debug mode by using the following command:

~~~
sudo /usr/share/packetbeat/bin/packetbeat -e -d "publish" -c /etc/packetbeat/packetbeat.yml
~~~